# 500 sample run results

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import camb
import emcee
import matplotlib as mpl
from matplotlib import pyplot as plt
import corner
import numpy as np
import mpl_toolkits.axes_grid1 as axgrid
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import minimize
from IPython.display import display, Math

plt.rcParams['mathtext.fontset'], plt.rcParams['font.family'] = 'stix', 'STIXGeneral'
plt.rcParams.update({'font.size': 17})

### Creating "fake" oberservational data

In [2]:
pars_test = camb.set_params(H0=67.5, 
                            ombh2=0.022, 
                            omch2=0.120*1.2, 
                            redshifts=[0], 
                            kmax=10, 
                            As=2e-9, 
                            ns=0.96, 
                            halofit_version='mead2020_feedback', 
                            HMCode_A_baryon = 0, 
                            HMCode_eta_baryon = 0, 
                            HMCode_logT_AGN = -10)
                            # zero the HMCode to get a universe without baryonic feedback
results_test = camb.get_transfer_functions(pars_test)
kh_test, _, pk_test = results_test.get_nonlinear_matter_power_spectrum(hubble_units=True, k_hunit=True)

kh_data = kh_test
pk_data = pk_test[0]

pk_err = 0.10*pk_data # creating error

## Using piecing method:

### Creating likelihood function using 'mead2020'

In [3]:
def log_likelihood_mead(theta, x, y, yerr): 
    # theta = [A, eta, logT_AGN] params, x --> kh_data, y --> pk_data, yerr --> pk_err
    A, eta, logT_AGN = theta
    model = camb.set_params(H0=67.5, ombh2=0.022, omch2=0.122, redshifts=[0], kmax=10, As=2e-9, ns=0.96, 
                           halofit_version='mead', 
                           HMCode_A_baryon = A, 
                           HMCode_eta_baryon = eta, 
                           HMCode_logT_AGN = logT_AGN)
    results = camb.get_transfer_functions(model)
    kh, _, pk = results.get_nonlinear_matter_power_spectrum(hubble_units=True, k_hunit=True)
    pk = pk[0]
    sigma2 = yerr**2
    return -0.5 * np.sum((y - pk)**2 / sigma2 + np.log(sigma2))

In [4]:
nll_mead = lambda *args: -log_likelihood_mead(*args) 
# maximizing the likelihood is the same as minimizing the negative of the liklihood
initial = np.array([3.13, 0.603, 7.8]) # using default HMCode values
soln_mead = minimize(nll_mead, initial, args=(kh_data, pk_data, pk_err))
A_ml_mead, eta_ml_mead, logT_AGN_ml_mead = soln_mead.x # maximum likelihood values
print('"mead" maximum likelihood values:', soln_mead.x)

"mead" maximum likelihood values: [ 3.14322588 -0.38471244  7.8       ]


### Creating likelihood function using 'mead2020_feedback'

In [ ]:
def log_likelihood_feedback(theta, x, y, yerr): 
    # theta = [A, eta, logT_AGN] params, x --> kh_data, y --> pk_data, yerr --> pk_err
    A, eta, logT_AGN = theta
    model = camb.set_params(H0=67.5, ombh2=0.022, omch2=0.122, redshifts=[0], kmax=10, As=2e-9, ns=0.96, 
                           halofit_version='mead2020_feedback', 
                           HMCode_A_baryon = A, 
                           HMCode_eta_baryon = eta, 
                           HMCode_logT_AGN = logT_AGN)
    results = camb.get_transfer_functions(model)
    kh, _, pk = results.get_nonlinear_matter_power_spectrum(hubble_units=True, k_hunit=True)
    pk = pk[0]
    sigma2 = yerr**2
    return -0.5 * np.sum((y - pk)**2 / sigma2 + np.log(sigma2))

In [ ]:
nll_feedback = lambda *args: -log_likelihood_feedback(*args) 
# maximizing the likelihood is the same as minimizing the negative of the liklihood
initial = np.array([3.13, 0.603, 7.8]) # using default HMCode values
soln_feedback = minimize(nll_feedback, initial, args=(kh_data, pk_data, pk_err))
A_ml_feedback, eta_ml_feedback, logT_AGN_ml_feedback = soln_feedback.x # maximum likelihood values
print('"mead2020_feedback" maximum likelihood values:', soln_feedback.x)

### Piecing maximum likelihood values together...

In [ ]:
pieced_ml = np.array([A_ml_mead, eta_ml_mead, logT_AGN_ml_feedback])
print('Pieced maximumum likelihood estimates:', pieced_ml)

### MCMC

In [ ]:
def log_prior(theta): # sets up priors on theta
    A, eta, logT_AGN = theta
    if -5.0 < A < 10.0 and -5.0 < eta < 5.0 and -20.0 < logT_AGN < 8:
        return 0.0
    return -np.inf

In [ ]:
def log_probability(theta, x, y, yerr):
    # theta = [A, eta, logT_AGN] params, x --> kh_data, y --> pk_data, yerr --> pk_err
    lp = log_prior(theta) 
    if not np.isfinite(lp): 
        return -np.inf
    return lp + log_likelihood_feedback(theta, x, y, yerr)

In [ ]:
pos = pieced_ml + 1e-4 * np.random.randn(32, 3) ## not sure what nwalkers value would be best
## would increading the number of walkers make it run faster?
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers,
                                ndim, 
                                log_probability, 
                                args=(kh_data, pk_data, pk_err))


sampler.run_mcmc(pos, 500, progress=True);

In [ ]:
samples = sampler.get_chain()
np.save('500_sample_test.npy', samples)

tau = sampler.get_autocorr_time()
print('Integrated autocorrelation time:', tau)

### Plotting and analyzing

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
labels = ['A_baryon', 'eta_baryon', 'logT_AGN']
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples)-1)
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel('step number');

In [ ]:
flat_samples = sampler.get_chain(discard=25, thin=12, flat=True) ## not sure about discard and thin values
print(flat_samples.shape)

fig = corner.corner(flat_samples, labels=labels, truths=initial);

# Not sure what to put into the 'truths' arg because I don't actually know what values of A, eta, and logT_AGN will
# reproduce the test observational data, so I made equal to HMCode defaults -- might be misunderstanding argument?

In [ ]:
labels = ['A-baryon', 'eta-baryon', 'logT_{AGN}']

for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.3f}}}^{{{2:.3f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))

### Quick plot for sanity check

In [ ]:
def baryon_feedback(A, eta, logT_AGN):
    pars = camb.set_params(H0=67.5, ombh2=0.022, omch2=0.122, redshifts=[0], kmax=10, As=2e-9, ns=0.96, 
                           halofit_version='mead2020_feedback', 
                           HMCode_A_baryon = A, 
                           HMCode_eta_baryon = eta, 
                           HMCode_logT_AGN = logT_AGN)
    results = camb.get_transfer_functions(pars)
    kh, _ , pk = results.get_nonlinear_matter_power_spectrum(hubble_units=True, k_hunit=True)
    return kh, pk[0]

In [ ]:
kh_model, pk_model = baryon_feedback(3.943, 0.330, -5.805)

plt.figure(figsize=(8,6))
plt.errorbar(kh_data, pk_data, yerr=pk_err, color='k', label='Observational data\n[omch2*1.2]')
plt.plot(kh_model, pk_model, "--g", label="Maximum likelihood")
plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1,1), loc='upper left')
plt.ylabel('$P_m(k)$ [($h^{-1}Mpc)^3$]')
plt.xlabel('Wavenumber, k [h/Mpc]');